# Imports

In [39]:
import cv2
import math
import numpy as np
import socket
import matplotlib.pyplot as plt
from cv2 import aruco
from scipy.interpolate import interp1d
mapper=interp1d([0,90],[0,255])
mapperd=interp1d([0,2000],[0,255])

# Aruco Markers Initialization

In [40]:
# Aruco Parameters
num_mark = 20 #Number of markers
size_mark = 500 #Size of markers


### Create Aruco markers 
dict_aruco = aruco.Dictionary_get(aruco.DICT_4X4_50)

front_marker_idx = 1
front_marker = aruco.drawMarker(dict_aruco, front_marker_idx, size_mark)
# cv2.imshow("fd ",front_marker)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
rear_marker_idx = 2
rear_marker = aruco.drawMarker(dict_aruco, rear_marker_idx, size_mark)
target1_marker_idx = 3
target2_marker_idx = 4
target3_marker_idx = 5
target1_marker = aruco.drawMarker(dict_aruco, target1_marker_idx, size_mark) 
target2_marker = aruco.drawMarker(dict_aruco, target2_marker_idx, size_mark)
target3_marker = aruco.drawMarker(dict_aruco, target3_marker_idx, size_mark)
# cv2.imshow("fjd ",target_1_marker)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
target_markers = [target1_marker, target2_marker, target3_marker]
target_marker_ids = [target1_marker_idx, target2_marker_idx, target3_marker_idx]

parameters = aruco.DetectorParameters_create()


# Helper Functions

In [41]:
def preprocess_corners(corners, ids):  
    ids = ids.tolist()
    if front_marker_idx in ids:
        front_corner_idx = ids.index(front_marker_idx)
        front_marker_centre = get_marker_centre(corners[front_corner_idx])
    else:
        front_marker_centre = None
                 
    if rear_marker_idx in ids:
        rear_corner_idx = ids.index(rear_marker_idx)
        rear_marker_centre = get_marker_centre(corners[rear_corner_idx])
    else:
        rear_marker_centre = None 
        
    if target1_marker_idx in ids:
        target1_corner_idx = ids.index(target1_marker_idx)
        target1_marker_centre = get_marker_centre(corners[target1_corner_idx])
    else:
        target1_marker_centre = None
  
        
    if target2_marker_idx in ids:
        target2_corner_idx = ids.index(target2_marker_idx)
        target2_marker_centre = get_marker_centre(corners[target2_corner_idx])
    else:
        target2_marker_centre = None
        
    if target3_marker_idx in ids:
        target3_corner_idx = ids.index(target3_marker_idx)
        target3_marker_centre = get_marker_centre(corners[target3_corner_idx])
    else:
        target3_marker_centre = None
        

    return [front_marker_centre, rear_marker_centre, target1_marker_centre, target2_marker_centre, target3_marker_centre]
            
def get_robot_centre(front_marker_centre, rear_marker_centre):
    return np.array([(front_marker_centre[0]+rear_marker_centre[0])/2 , (front_marker_centre[1]+rear_marker_centre[1])/2])


def get_marker_centre(corner):
    cornerUL = corner[0][0]
    cornerUR = corner[0][1]
    cornerBR = corner[0][2]
    cornerBL = corner[0][3]
    return np.array([ (cornerUL[0]+cornerBR[0])/2 , (cornerUL[1]+cornerBR[1])/2 ])

# def get_vehicle_horizontal_orientation(left_wheel_pos, right_wheel_pos):


# def get_robot_heading(front_centre, rear_centre):
#     return front_centre - rear_centre # as np array

### Corrected: robot heading is the direction or angle
def get_robot_heading(front_centre, rear_centre):
    return get_vector_angle(front_centre - rear_centre) # as np array

def get_target_vector(robot_centre, target_centre):
    return target_centre - robot_centre # as np array

def get_vector_angle(vector):
    return math.atan2(vector[1],vector[0])

def get_heading_error(robot_heading, target_angle):
    return target_angle - robot_heading

def get_rotation_direction(heading_error):
    #mode 1 rotates counter clockwise
    #mode -1 rotates clockwise
    if abs(heading_error) <= 90:
        return int(np.sign(heading_error) )
    return - int(np.sign(heading_error))

def get_abs_heading_error(heading_error):
    if abs(heading_error) <= 90:
        return abs(heading_error)
    return 180-abs(heading_error) 

def get_distance_to_target(robot_centre, target_centre):
    return np.linalg.norm(get_target_vector(robot_centre, target_centre))

def get_longitudinal_direction(heading_error):
    if abs(heading_error) <= 90:
        return 0 #mode 0 moves forward
    return 2 #mode 2 moves backwards

# Communication initialization

In [19]:

# Replace with the IP address of the ESP32
UDP_IP = "192.168.43.57"
UDP_PORT = 8080
# Create a UDP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Main()

In [ ]:
### Load the video
# cap = cv2.VideoCapture(0)
PWM1=0
PWM2=0
mode=0
#to be tuned
kpr=0.8
kpd=0.8
kpoff=0.8
while True:
    # Capture a frame
    ret, frame = cap.read()
#     frame = cv2.imread('scenario.png')
    
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Original Image', gray)
    cv2.waitKey(1)
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, dict_aruco, parameters=parameters)
    list_ids = np.ravel(ids)
    
    front_marker_centre, rear_marker_centre, target1_marker_centre, \
     target2_marker_centre, target3_marker_centre = preprocess_corners(corners, list_ids)
    #TODO user input
    if  front_marker_centre is None or  rear_marker_centre is None or target1_marker_centre is None:
        continue
    target_centre = target1_marker_centre
    robot_centre= get_robot_centre(front_marker_centre, rear_marker_centre)
    robot_heading= get_robot_heading(front_marker_centre, rear_marker_centre)
    target_vector=get_target_vector(robot_centre, target_centre)
    target_angle=get_vector_angle(target_vector)
    heading_error=get_heading_error(robot_heading, target_angle)
    absHeadingError=get_abs_heading_error(heading_error)
    
    # Drawing on frame
    cv2.arrowedLine(frame, tuple(robot_centre.astype(int)), tuple(target_centre.astype(int)), (255,0,0), thickness=3) 
    cv2.arrowedLine(frame, tuple(robot_centre.astype(int)), tuple(front_marker_centre.astype(int)), (255,0,0), thickness=3) 
    
    cv2.imshow('frame', frame)
    cv2.waitKey(1)
    
    
    if (absHeadingError>5):
        #rotate around itslef
        controledAngleError = kpr * absHeadingError
        PWM1 = mapper(controledAngleError)
        PWM2 = PWM1
        mode = get_rotation_direction(heading_error)
        
    else:
        #Move to target
        errord = get_distance_to_target(robot_centre, target_centre)
        if(errord<50):
            avgSpeed = 0
        else:
            avgSpeed = mapperd(kpd*errord)
        if(absHeadingError < 2):
            offset = 0
        else:
            offset = kpoff * absHeadingError
        mode = get_longitudinal_direction(heading_error)
        rotationDirection = get_rotation_direction(heading_error)
        # at mode zero(Forwards) PWM1 Left------PWM2 Right
        #at mode 2 (Backwards) PWM1 Right --------- PWM2 Left
        if mode==0:
            if(rotationDirection == 1):
                PWM2 = avgSpeed + offset
                PWM1 = avgSpeed - offset
            else:
                PWM2 = avgSpeed - offset
                PWM1 = avgSpeed + offset  
        else:
            if(rotationDirection == -1):
                PWM2 = avgSpeed + offset
                PWM1 = avgSpeed - offset
            else:
                PWM2 = avgSpeed - offset
                PWM1 = avgSpeed + offset
    print(PWM1,PWM2,mode,end="\r")
    message = str(PWM1) + ',' + str(PWM2) + ',' + str(mode)
    ###
        sock.sendto(data, (UDP_IP, UDP_PORT))
        bytesAddressPair = sock.recvfrom(1024)
        message = bytesAddressPair[0].decode()
        address = bytesAddressPair[1]
        print(message)
    ###
    
    if cv2.waitKey(1) == 13: #13 Enter Key
        break
    #send signal to arduino
#      if list_ids[0] == 5:
#         data = str(0).encode()
#     elif avg_intensity > 80:
#         data = str(1).encode()
#     else:
#         data = str(-1).encode()
    
#     sock.sendto(data, (UDP_IP, UDP_PORT))
    
#     bytesAddressPair = sock.recvfrom(1024)

#     message = bytesAddressPair[0].decode()
#     address = bytesAddressPair[1]
#     print(message)
     #,end="\r"


cap.release()                                             # For releasing cap and out. 
cv2.destroyAllWindows()
